In [1]:
from config import config, key
from train import train
import wandb

C:\Users\KaranjotVendal\mambaforge\envs\thesis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\KaranjotVendal\mambaforge\envs\thesis\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
import wandb
wandb.login(key=key.wandb_key)

wandb: Currently logged in as: karanjot. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\KaranjotVendal/.netrc


True

In [3]:
wandb.init(
project=f"{config.MODEL}", 
name=f"experiment_{config.MOD}",
notes=f'establishing baseline for {config.MODEL}_{config.MOD}.(Convx:LSTM:FC:classifier)',
config={
"learning_rate": config.LEARNING_RATE,
"architecture": "CNN-GRU-MASK-FC-Classifier",
"dataset": "MICAA MRI",
"epochs": config.NUM_EPOCHS,
"BATCH_SIZE": config.BATCH_SIZE,
"Image Size": config.IMG_SIZE,
})

In [4]:
#mod = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
#for m in mod:
train()

--------------FOLD:1-----------------------
 Train Epoch: 1/25 | Loss: 0.68918 | Accuracy: 0.5226% | F1 Score: 0.5210 | AUROC: 0.5336 | Time: 75.798166513443
Checkpoint saved at ./checkpoints/Res50GRU_model_FLAIR_1.pth | Test acc: 0.58% | Test F1: 0.571% | Best AUROC: 0.627
Validation Epoch: 0m 19s | Accuracy: 0.58% | F1 Score: 0.5710 | AUROC: 0.6268
 Train Epoch: 2/25 | Loss: 0.67383 | Accuracy: 0.5828% | F1 Score: 0.5822 | AUROC: 0.6146 | Time: 75.26824235916138
Validation Epoch: 0m 19s | Accuracy: 0.56% | F1 Score: 0.5595 | AUROC: 0.5647
 Train Epoch: 3/25 | Loss: 0.66498 | Accuracy: 0.5699% | F1 Score: 0.5686 | AUROC: 0.6313 | Time: 75.31661701202393
Validation Epoch: 0m 19s | Accuracy: 0.57% | F1 Score: 0.5634 | AUROC: 0.5852
 Train Epoch: 4/25 | Loss: 0.64484 | Accuracy: 0.6086% | F1 Score: 0.6086 | AUROC: 0.6953 | Time: 75.42324566841125
Validation Epoch: 0m 19s | Accuracy: 0.56% | F1 Score: 0.5621 | AUROC: 0.5790
 Train Epoch: 5/25 | Loss: 0.62626 | Accuracy: 0.6366% | F1 Score

Avg performance,▁
Std f1 score,▁
test AUROC,▆▃█▂▁
test acc,▅▁█▁▂
test f1,▆▁█▂▅
train AUROC,▂▄▆▇▇███▁▄▆▇████▁▅▇▇████▃▄▆▇████▃▅▇▇████
train acc,▁▃▅▆▇▇██▂▃▅▆▇▇██▁▄▅▆▇▇██▂▃▅▆▇▇▇█▃▄▆▆▇▇██
train f1_score,▂▃▆▆▇▇██▁▃▅▆▇▇██▂▅▆▇▇███▃▄▅▆▇▇▇█▃▄▆▇▇▇██
train loss,█▇▆▅▄▃▂▂█▇▆▅▃▂▂▁█▇▆▅▄▃▂▁█▇▆▅▄▃▂▁▇▇▅▄▃▂▂▁
val AUROC,▇▆▆▆▆▆▆▆▅▄▂▂▂▁▁▂█▂▄▄▅▄▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
val acc,▅▅▅▆▅▄▅▅▃▃▄▃▃▃▂▃█▁▃▃▄▂▄▄▄▄▄▄▄▄▄▄▃▄▄▄▄▄▃▅
